[View in Colaboratory](https://colab.research.google.com/github/pgurazada/ml-projects/blob/master/2018-06-13_mooc-analysis.ipynb)

### Connect Google Drive

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

In [ ]:
import getpass

In [ ]:
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
!ls drive/data/moocs

### Design the MLP

In [ ]:
!pip install -U feather-format

In [1]:
import pandas as pd
import numpy as np

In [2]:
from keras.models import Sequential

from keras.layers import Dense, Activation

from keras.losses import binary_crossentropy

from keras.optimizers import RMSprop

from keras.metrics import binary_accuracy

from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from keras.wrappers.scikit_learn import KerasClassifier

from keras import backend as K

C:\Users\kimmcodxb\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [54]:
from sklearn.metrics import f1_score

### Assembling the neural net for the largest data set

In [4]:
cs50x_features_train = pd.read_feather('processed-final/HarvardXCS50x2012_features_train.feather')
cs50x_features_test = pd.read_feather('processed-final//HarvardXCS50x2012_features_test.feather')
cs50x_labels_train = pd.read_feather('processed-final//HarvardXCS50x2012_labels_train.feather')
cs50x_labels_test = pd.read_feather('processed-final//HarvardXCS50x2012_labels_test.feather')

In [5]:
features_train, features_test = cs50x_features_train.drop('index', axis=1), cs50x_features_test.drop('index', axis=1)
labels_train, labels_test = cs50x_labels_train.drop('index', axis=1), cs50x_labels_test.drop('index', axis=1)

features_train = np.array(features_train)
features_test = np.array(features_test)

labels_train = np.array(labels_train).ravel()
labels_test = np.array(labels_test).ravel()

In [6]:
features_train.shape

(135696, 19)

In [7]:
labels_train.shape

(135696,)

In [59]:
def build(network_type=Sequential, 
          nb_initial_layer=64,
          dense_layer_lst=[64],
          nb_final_layer=1,
          learning_rate=1e-4):
    model = network_type()
    
    model.add(Dense(nb_initial_layer, input_shape=(features_train.shape[1], )))
    model.add(Activation('relu'))
    
    for nb_units in dense_layer_lst:
        model.add(Dense(nb_units))
        model.add(Activation('relu'))
        
    model.add(Dense(nb_final_layer))
    model.add(Activation('sigmoid'))
  
  model.compile(optimizer=RMSprop(lr=learning_rate),
                loss=binary_crossentropy,
                metrics=[binary_accuracy])
  
  return model

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)

In [51]:
K.clear_session()

In [52]:
model = build(dense_layer_lst=[32, 32, 32])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                1280      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
__________

In [53]:
model_output = model.fit(features_train, labels_train,
                         batch_size=128,
                         epochs=20,
                         validation_split=0.2,
                         callbacks=[EarlyStopping(patience=4), ReduceLROnPlateau(patience=4, min_lr=1e-6)])

Train on 108556 samples, validate on 27140 samples
Epoch 1/20
108556/108556 [==============================] - 2s 15us/step - loss: 1.6528 - binary_accuracy: 0.4596 - val_loss: 0.8090 - val_binary_accuracy: 0.5302
Epoch 2/20
108556/108556 [==============================] - 1s 12us/step - loss: 0.7907 - binary_accuracy: 0.5999 - val_loss: 1.1002 - val_binary_accuracy: 0.5567
Epoch 3/20
108556/108556 [==============================] - 1s 12us/step - loss: 0.7105 - binary_accuracy: 0.6360 - val_loss: 0.7565 - val_binary_accuracy: 0.6714
Epoch 4/20
108556/108556 [==============================] - 1s 13us/step - loss: 0.6566 - binary_accuracy: 0.6606 - val_loss: 0.6444 - val_binary_accuracy: 0.6466
Epoch 5/20
108556/108556 [==============================] - 1s 14us/step - loss: 0.6372 - binary_accuracy: 0.6665 - val_loss: 0.6198 - val_binary_accuracy: 0.6649
Epoch 6/20
108556/108556 [==============================] - 2s 15us/step - loss: 0.6272 - binary_accuracy: 0.6686 - val_loss: 0.6811 -

In [55]:
f1_score(model.predict(features_test), labels_test)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [58]:
model.predict_proba(features_test)

C:\Users\kimmcodxb\Anaconda3\lib\site-packages\keras\models.py:1153: UserWarning: Network returning invalid probability values. The last layer might not normalize predictions into probabilities (like softmax or sigmoid would).
  warnings.warn('Network returning invalid probability values. '


array([[0.74352163],
       [0.15976715],
       [0.3160963 ],
       ...,
       [0.7567098 ],
       [0.7775063 ],
       [0.93345547]], dtype=float32)